In [1]:
# ライブラリをインポート
import numpy as np
import pandas as pd

import streamlit as st
import yfinance as yf
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ta
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands

C:\Users\msg22\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
# 各関数を定義
# 株価を取得する関数
def get_stock_price(ticker, period, interval):
    data = yf.download(ticker, period = period, interval = interval)
    return data


# 株価データを描画する関数
def plot_stock_price(ticker, period, title):
    data = get_stock_price(ticker, period = period, interval = '1d')
    # プロットの区切りを設定
    fig = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.03)

    # 株価データと移動曲線を描画
    fig.add_trace(go.Scatter(x=data.index, y=data['Close'], mode='lines', name='original'))
    fig.add_trace(go.Scatter(x=data.index, y=data['Close'].rolling(window=50).mean(), name='MA50', line=dict(color='lightblue')))
    fig.add_trace(go.Scatter(x=data.index, y=data['Close'].rolling(window=75).mean(), name='MA75', line=dict(color='lightsalmon')))

    # ボリンジャーバンドを描画
    indicator_bb = BollingerBands(close=data["Close"], window=20, window_dev=2)
    fig.add_trace(go.Scatter(x=data.index, y=indicator_bb.bollinger_hband(), name='Upper BB', line=dict(color='palevioletred', dash='dash')))
    fig.add_trace(go.Scatter(x=data.index, y=indicator_bb.bollinger_lband(), name='Lower BB', line=dict(color='palevioletred', dash='dash')))

    fig.update_layout(title={'text': title, 'x': 0.5, 'y': 0.8, 'xanchor': 'center', 'yanchor': 'top'}, xaxis_rangeslider_visible=False, showlegend=False)
    return fig

In [3]:
# 株価を取得したい銘柄を指定
################
ticker = '7012.T'
period = '2y'
interval = '1d'
################

In [4]:
# ディレクトリを指定
input_data = 'data/'

# データの読み込み
rakuten = pd.read_excel(input_data + '02_運用_rakuten.xlsx', sheet_name = '国内株式_特定')
# 銘柄コード列を文字列に変換
rakuten['銘柄コード'] = rakuten['銘柄コード'].astype(str)
rakuten

,約定日,受渡日,銘柄コード,銘柄名,口座区分,取引区分,売買区分,数量［株］,単価［円］,手数料［円］,税金等［円］,税区分,受渡金額［円］,実現損益[円]
0,2021-01-20,2021-01-22,2503,キリンＨＤ,特定,現物,買付,100,2265.9,0,0,-,226590.0,0.0
1,2021-01-21,2021-01-25,2503,キリンＨＤ,特定,現物,売付,100,2306.4,0,0,源徴あり,230640.0,4050.0
2,2021-01-21,2021-01-25,3395,サンマルクホールディングス,特定,現物,買付,100,1490.0,0,0,-,149000.0,0.0
3,2021-01-25,2021-01-27,4188,三菱ケミカルＨＬＤＧＳ,特定,現物,買付,100,745.3,0,0,-,74530.0,0.0
4,2021-01-25,2021-01-27,7616,コロワイド,特定,現物,買付,100,1817.0,0,0,-,181700.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570,2025-02-03,2025-02-05,5401,日本製鉄,特定,現物,買付,100,3184.0,0,0,-,318400.0,0.0
571,2025-02-03,2025-02-05,8316,三井住友フィナンシャルG,特定,現物,買付,100,3756.0,0,0,-,375600.0,0.0
572,2025-02-04,2025-02-06,7867,タカラトミー,特定,現物,売付,100,4241.0,0,0,源徴あり,424100.0,18800.0
573,2025-02-04,2025-02-06,8316,三井住友フィナンシャルG,特定,現物,売付,100,3787.0,0,0,源徴あり,378700.0,3100.0
